# Create unified excel file

In [ ]:
# import packages
import numpy as np
import pandas as pd
from datetime import datetime
import os
import time
import open3d as o3d
import pickle
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
all_EEG_10_20_landmark_names = ['Fp1', 'Fp2',
                                'F7', 'F3', 'Fz', 'F4', 'F8',
                                'A1', 'T3', 'C3', 'Cz', 'C4', 'T4', 'A2',
                                'T5', 'P3', 'Pz', 'P4', 'T6',
                                'O1', 'O2']

all_facial_landmark_names = ['1', '2', '3', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26',
                             '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41',
                             '42', '43', '44', '45', '46', '47', '48']

all_landmarks_names = [all_EEG_10_20_landmark_names, all_facial_landmark_names, ['nasion', 'inion']]
all_landmarks_names = [current_name for current_names_group in all_landmarks_names for current_name in current_names_group]

In [ ]:
datasets_folder = './MRI_datasets/'
dataset_folder = datasets_folder+'ADNI/' # 'IXI/' / 'ADNI/'
data_folder = dataset_folder+'Data_Chamfer_10/'
all_subjects_folders = os.listdir(data_folder)

for i, current_filename in enumerate(all_subjects_folders):
    if ".xlsx" not in current_filename:
        del all_subjects_folders[i]

all_subjects_folders.sort()

In [ ]:
for current_subject_index, current_subject_filename in enumerate(all_subjects_folders):
    print(f"Started {current_subject_filename}, {current_subject_index+1}/{len(all_subjects_folders)}")
    current_subject = current_subject_filename.split('.')[0]
    
    current_subject_dataframe = pd.ExcelFile(data_folder+current_subject_filename)
    current_sheet_names = current_subject_dataframe.sheet_names
    current_num_of_sheets = len(current_sheet_names)
    
    skin_coordinates_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Skin coordinates')
    skin_original_coordinates_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Skin original coordinates')
    skin_normals_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Skin normals')
    skin_original_normals_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Skin original normals')
    shape_coordinates_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Shape coordinates')
    skin_geodesic_distances_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Skin distances')
    inverse_matrices_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Inverse transformations')
    stats_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Stats')
    
    current_subject_skin_coordinates = pd.read_excel(data_folder+current_subject_filename, sheet_name=skin_coordinates_index, index_col=0)
    current_subject_skin_original_coordinates = pd.read_excel(data_folder+current_subject_filename, sheet_name=skin_original_coordinates_index, index_col=0)

    current_subject_skin_normals = pd.read_excel(data_folder+current_subject_filename, sheet_name=skin_normals_index, index_col=0)
    current_subject_skin_original_normals = pd.read_excel(data_folder+current_subject_filename, sheet_name=skin_original_normals_index, index_col=0) 
    current_subject_shape_coordinates = pd.read_excel(data_folder+current_subject_filename, sheet_name=shape_coordinates_index, index_col=0)    
    current_subject_skin_geodesic_distances = pd.read_excel(data_folder+current_subject_filename, sheet_name=skin_geodesic_distances_index, index_col=0)
    current_subject_inverse_matrices = pd.read_excel(data_folder+current_subject_filename, sheet_name=inverse_matrices_index, index_col=0)
    current_subject_stats = pd.read_excel(data_folder+current_subject_filename, sheet_name=stats_index, index_col=0)
    
    if 'mean_EEG_10_20_landmarks_distance' in current_subject_stats.index:
        current_subject_stats.drop(index='mean_EEG_10_20_landmarks_distance', inplace=True)
    
    if current_subject_index==0:
        current_subject_skin_coordinates = current_subject_skin_coordinates.rename(columns={"indices": f"{current_subject}_indices",
                                                                                            "x": f"{current_subject}_x",
                                                                                            "y": f"{current_subject}_y",
                                                                                            "z": f"{current_subject}_z",
                                                                                           })
        
        current_subject_skin_original_coordinates = current_subject_skin_original_coordinates.rename(columns={"indices": f"{current_subject}_indices",
                                                                                                              "x": f"{current_subject}_x",
                                                                                                              "y": f"{current_subject}_y",
                                                                                                              "z": f"{current_subject}_z",
                                                                                                             })

        current_subject_skin_normals = current_subject_skin_normals.rename(columns={"x": f"{current_subject}_x",
                                                                                    "y": f"{current_subject}_y",
                                                                                    "z": f"{current_subject}_z",
                                                                                   })
        
        current_subject_skin_original_normals = current_subject_skin_original_normals.rename(columns={"x": f"{current_subject}_x",
                                                                                                      "y": f"{current_subject}_y",
                                                                                                      "z": f"{current_subject}_z",
                                                                                                     })


        current_subject_shape_coordinates = current_subject_shape_coordinates.rename(columns={"indices": f"{current_subject}_indices",
                                                                                              "x": f"{current_subject}_x",
                                                                                              "y": f"{current_subject}_y",
                                                                                              "z": f"{current_subject}_z",
                                                                                             })
        
        current_subject_inverse_matrices = current_subject_inverse_matrices.rename(columns={0: f"{current_subject}_0",
                                                                                 1: f"{current_subject}_1",
                                                                                 2: f"{current_subject}_2",
                                                                                 3: f"{current_subject}_3",
                                                                                })
               
        current_subject_stats = current_subject_stats.rename(columns={"value": f"{current_subject}"})
                       
        all_subjects_pd = []
        all_subjects_pd.append(current_subject_skin_coordinates)
        all_subjects_pd.append(current_subject_skin_original_coordinates)
        all_subjects_pd.append(current_subject_skin_normals)
        all_subjects_pd.append(current_subject_skin_original_normals)
        all_subjects_pd.append(current_subject_shape_coordinates)
        all_subjects_pd.append(current_subject_skin_geodesic_distances)
        all_subjects_pd.append(current_subject_inverse_matrices)
        all_subjects_pd.append(current_subject_stats)
        
        all_subjects_indices = []
        for i in range(len(all_subjects_pd)):
            all_subjects_indices.append(list(all_subjects_pd[i].index))
    else:
        all_subjects_pd = all_subjects_pd.copy()        
        
        current_subject_pd = []
        current_subject_pd.append(current_subject_skin_coordinates)
        current_subject_pd.append(current_subject_skin_original_coordinates)
        current_subject_pd.append(current_subject_skin_normals)
        current_subject_pd.append(current_subject_skin_original_normals)
        current_subject_pd.append(current_subject_shape_coordinates)
        current_subject_pd.append(current_subject_skin_geodesic_distances)
        current_subject_pd.append(current_subject_inverse_matrices)
        current_subject_pd.append(current_subject_stats)
        
        current_subject_indices = []
        for i in range(len(current_subject_pd)):
            current_subject_indices.append(list(current_subject_pd[i].index))
        
        for i in range(len(all_subjects_indices)):
            if current_subject_pd[i].values.shape[1]==4 and i<len(all_subjects_indices)-2: # coordinates
                if len(all_landmarks_names)==len(current_subject_indices[i]):
                    current_subject_sheet_data = current_subject_pd[i].values
                else:
                    if i==4: # shape doesn't include inion and nasion landmarks
                        current_landmarks_names = all_landmarks_names[:-2]
                    else:
                        current_landmarks_names = all_landmarks_names
                    
                    current_subject_sheet_data = np.zeros((len(current_landmarks_names),
                                                                 current_subject_pd[i].values.shape[1]))
                    
                    current_set = set(current_subject_indices[i])
                    relevant_rows = [j for j, current_landmark_name in enumerate(current_landmarks_names) 
                                     if current_landmark_name in current_set]
                    
                    current_subject_sheet_data[relevant_rows, :] = current_subject_pd[i].values
                                                       
                all_subjects_pd[i] = pd.concat([all_subjects_pd[i], pd.Series(current_subject_sheet_data[:, 0], name=f"{current_subject}_indices", index=all_subjects_pd[i].index)], axis=1)
                all_subjects_pd[i] = pd.concat([all_subjects_pd[i], pd.Series(current_subject_sheet_data[:, 1], name=f"{current_subject}_x", index=all_subjects_pd[i].index)], axis=1)
                all_subjects_pd[i] = pd.concat([all_subjects_pd[i], pd.Series(current_subject_sheet_data[:, 2], name=f"{current_subject}_y", index=all_subjects_pd[i].index)], axis=1)
                all_subjects_pd[i] = pd.concat([all_subjects_pd[i], pd.Series(current_subject_sheet_data[:, 3], name=f"{current_subject}_z", index=all_subjects_pd[i].index)], axis=1)
                    
            
            elif current_subject_pd[i].values.shape[1]==3: # normals
                current_landmarks_names = all_landmarks_names
                current_subject_sheet_data = np.zeros((len(current_landmarks_names),
                                                             current_subject_pd[i].values.shape[1]))
                current_set = set(current_subject_indices[i])
                relevant_rows = [j for j, current_landmark_name in enumerate(current_landmarks_names) 
                                 if current_landmark_name in current_set]
                
                current_subject_sheet_data[relevant_rows, :] = current_subject_pd[i].values
                    
                all_subjects_pd[i] = pd.concat([all_subjects_pd[i], pd.Series(current_subject_sheet_data[:, 0], name=f"{current_subject}_x", index=all_subjects_pd[i].index)], axis=1)
                all_subjects_pd[i] = pd.concat([all_subjects_pd[i], pd.Series(current_subject_sheet_data[:, 1], name=f"{current_subject}_y", index=all_subjects_pd[i].index)], axis=1)
                all_subjects_pd[i] = pd.concat([all_subjects_pd[i], pd.Series(current_subject_sheet_data[:, 2], name=f"{current_subject}_z", index=all_subjects_pd[i].index)], axis=1)

            
            elif current_subject_pd[i].values.shape[1]==1: # geodesic distances and stats
                all_subjects_pd[i] = pd.concat([all_subjects_pd[i], pd.Series(current_subject_pd[i].values.squeeze(), name=f"{current_subject}", index=all_subjects_pd[i].index)], axis=1)
                
            else:
                current_df = pd.DataFrame(current_subject_pd[i].values.squeeze(),
                                          columns=[f"{current_subject}_0", f"{current_subject}_1", f"{current_subject}_2", f"{current_subject}_3"])
                all_subjects_pd[i] = pd.concat([all_subjects_pd[i], current_df], axis=1)

## Save unified excel file

In [ ]:
if 1:
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(dataset_folder+'Dataset_Chamfer.xlsx')

    # Write each dataframe to a different worksheet.
    all_subjects_pd[0].to_excel(writer, sheet_name='Skin coordinates')
    all_subjects_pd[1].to_excel(writer, sheet_name='Skin original coordinates')
    
    all_subjects_pd[2].to_excel(writer, sheet_name='Skin normals')
    all_subjects_pd[3].to_excel(writer, sheet_name='Skin original normals')
    all_subjects_pd[4].to_excel(writer, sheet_name='Shape coordinates')
    all_subjects_pd[5].to_excel(writer, sheet_name='Skin distances')
    all_subjects_pd[6].to_excel(writer, sheet_name='Inverse transformations')
    all_subjects_pd[7].to_excel(writer, sheet_name='Stats')

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()